In [ ]:
import cv2
from math import sqrt
import numpy as np 
from matplotlib import pyplot as plt

In [ ]:
height = 500
width = 500

imgBlank = np.zeros((height,width, 3), np.uint)
url = "http://192.168.6.250:8080/video"
cap = cv2.VideoCapture(url)

In [ ]:
def getContours(img, original_img):
    contours,hierachy = cv2.findContours(img, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area>60000:
            cv2.drawContours(original_img,cnt,-1,(0,255,0),3)
    
def biggestContour(img,original_img):
    contours,hierachy = cv2.findContours(img, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    biggest = np.array([])
    max_area = 0
    for cnt in contours:
        area= cv2.contourArea(cnt)
        if area>60000:
            # cv2.drawContours(original_img,cnt,-1,(0,255,0),3)
            peri = cv2.arcLength(cnt,True)
            approx = cv2.approxPolyDP(cnt, 0.02*peri,True)
            if area>max_area and len(approx)==4:
                max_area = area
                biggest = approx
    return biggest, max_area

def reorder(points):
    points = points.reshape((4,2))
    pointsnew = np.zeros((4,1,2),dtype= np.int32)
    add = points.sum(1)
    pointsnew[0] = points[np.argmin(add)]
    pointsnew[3] = points[np.argmax(add)]
    diff = np.diff(points,axis=1)
    pointsnew[1] = points[np.argmin(diff)]
    pointsnew[2] = points[np.argmax(diff)]
    return pointsnew

def splitBoxes(img,n):
    rows = np.vsplit(img,n)
    boxes=[]
    for r in rows:
        cols = np.hsplit(r,n)
        for box in cols:
            boxes.append(box)
    return boxes

col_val = [[0,0,254,1],[0,141,2,2],[4,224,234,3],[254,41,14,4],[0,137,250,5]]

def predict(boxes):
    board = []
    for image in boxes:
        col = (image[50,50])
        e = 0
        for i in col_val:
            if i[0]==col[0] and i[1]==col[1] and i[2]==col[2]:
                e = i[3]
        board.append(e)
    return board

In [ ]:
while True:
    camera, img = cap.read()
    img = cv2.resize(img,(500,500))
    imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    imgBlur = cv2.GaussianBlur(imgGray,(3,3),0)
    imgCanny = cv2.Canny(imgBlur,50,200)
    img_copy = img.copy()

    biggest, maxArea = biggestContour(imgCanny,img_copy)

    if biggest.size!= 0:
        biggest = reorder(biggest)
        cv2.drawContours(img_copy, biggest, -1,(0,255,0),10)
        pts1 = np.float32(biggest)
        pts2 = np.float32([[0,0],[width,0],[0,height],[width,height]])
        matrix = cv2.getPerspectiveTransform(pts1,pts2)
        imgWarpColored = cv2.warpPerspective(img, matrix, (width,height))
        imgDetectedCircles = imgBlank.copy()
        # imgWarpColored = cv2.cvtColor(imgWarpColored,cv2.COLOR_BGR2GRAY)
    
    cv2.imshow('Webcam', img)
    if cv2.waitKey(1) & 0xff == ord('q'):
        cv2.destroyAllWindows()
        break